#### Data Science Captstone - Week 2

Import panadas and create beautiful soup if we need it 

In [8]:
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup
import requests

In [9]:
df8 = pd.read_csv('neighborhoods2.csv') # neighborhoods 2 contains the latitude and longitude of our three neighborhoods in DC 

In [10]:
df8.head() # let's show them

,Neighborhood,Latitude,Longitude
0,U Street Corridor,38.9171,-77.0291
1,Logan Circle,38.9096,-77.0296
2,Columbia Heights,38.9288,-77.0305


In [11]:
type(df8) # verifying dataframe

pandas.core.frame.DataFrame

#### Now let's take a look at the DC Maps
Import first

In [12]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

print('Libraries imported.')

Solving environment: done

## Package Plan ##

  environment location: /home/jupyterlab/conda

  added / updated specs: 
    - geopy


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    geopy-1.17.0               |             py_0          49 KB  conda-forge
    geographiclib-1.49         |             py_0          32 KB  conda-forge
    ------------------------------------------------------------
                                           Total:          82 KB

The following NEW packages will be INSTALLED:

    geographiclib: 1.49-py_0   conda-forge
    geopy:         1.17.0-py_0 conda-forge


geopy-1.17.0         | 49 KB     | ##################################### | 100% 
geographiclib-1.49   | 32 KB     | ##################################### | 100% 
Preparing transaction: done
Verifying transaction: done
Executing transaction: done
Solving environment: done

## Package Plan ##

  envi

In [13]:
address = 'Washington'  # following the NYC lab just to see what happens

geolocator = Nominatim()
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Washington are {}, {}.'.format(latitude, longitude))

/home/jupyterlab/conda/lib/python3.6/site-packages/geopy/geocoders/osm.py:143: UserWarning: Using Nominatim with the default "geopy/1.17.0" `user_agent` is strongly discouraged, as it violates Nominatim's ToS https://operations.osmfoundation.org/policies/nominatim/ and may possibly cause 403 and 429 HTTP errors. Please specify a custom `user_agent` with `Nominatim(user_agent="my-application")` or by overriding the default `user_agent`: `geopy.geocoders.options.default_user_agent = "my-application"`. In geopy 2.0 this will become an exception.
  UserWarning


The geograpical coordinate of Washington are 38.8950092, -77.0365625.


In [14]:
# create map of New York using latitude and longitude values
map_washington = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, neighborhood in zip(df8['Latitude'], df8['Longitude'], df8['Neighborhood']):
    label = '{}'.format(neighborhood)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_washington)  
    
map_washington

In [15]:
washington_data = df8
washington_data.head()

,Neighborhood,Latitude,Longitude
0,U Street Corridor,38.9171,-77.0291
1,Logan Circle,38.9096,-77.0296
2,Columbia Heights,38.9288,-77.0305


#### Define Foursquare Credentials and Version

In [16]:
CLIENT_ID = 'WJSQRHGQU0GVDHLEMW2IOS23DBIOMACV2X5VBVCWMV11S1H3' # your Foursquare ID
CLIENT_SECRET = 'NB1WGADMKQ2NPNHBF51BZL3GBDGLSQ15Z1NDPCX25JIHQMOB' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: WJSQRHGQU0GVDHLEMW2IOS23DBIOMACV2X5VBVCWMV11S1H3
CLIENT_SECRET:NB1WGADMKQ2NPNHBF51BZL3GBDGLSQ15Z1NDPCX25JIHQMOB


In [17]:
washington_data.loc[0, 'Neighborhood'] # testing out 

'U Street Corridor'

In [18]:
neighborhood_latitude = washington_data.loc[0, 'Latitude'] # neighborhood latitude value
neighborhood_longitude = washington_data.loc[0, 'Longitude'] # neighborhood longitude value

neighborhood_name = washington_data.loc[0, 'Neighborhood'] # neighborhood name

print('Latitude and longitude values of {} are {}, {}.'.format(neighborhood_name, 
                                                               neighborhood_latitude, 
                                                               neighborhood_longitude))

Latitude and longitude values of U Street Corridor are 38.9171, -77.0291.


In [19]:
LIMIT = 100 # limit of number of venues returned by Foursquare API
radius = 500 # define radius


In [20]:
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    neighborhood_latitude, 
    neighborhood_longitude, 
    radius, 
    LIMIT)
url # display URL

'https://api.foursquare.com/v2/venues/explore?&client_id=WJSQRHGQU0GVDHLEMW2IOS23DBIOMACV2X5VBVCWMV11S1H3&client_secret=NB1WGADMKQ2NPNHBF51BZL3GBDGLSQ15Z1NDPCX25JIHQMOB&v=20180605&ll=38.9171,-77.0291&radius=500&limit=100'

In [21]:
results = requests.get(url).json()  # the results are long so I didn't show them

In [22]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [23]:
venues = results['response']['groups'][0]['items']
    
nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head(40) # looking at all of the venues on U Street

,name,categories,lat,lng
0,Ben's Chili Bowl,Hot Dog Joint,38.917320,-77.028760
1,&pizza,Pizza Place,38.916755,-77.029245
2,Izakaya Seki,Japanese Restaurant,38.918151,-77.027777
3,U Street Music Hall,Music Venue,38.917145,-77.027727
4,Busboys and Poets,American Restaurant,38.917909,-77.031680
5,The Fainting Goat,Gastropub,38.916834,-77.030924
6,The Lincoln Theatre,Concert Hall,38.917411,-77.028942
7,Jeni's Splendid Ice Creams,Ice Cream Shop,38.916381,-77.031877
8,Marvin,Bar,38.917116,-77.031713
9,The Smith,American Restaurant,38.916838,-77.030565


In [24]:
print('{} venues were returned by Foursquare.'.format(nearby_venues.shape[0]))

100 venues were returned by Foursquare.


In [25]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [26]:
washington_venues = getNearbyVenues(names=washington_data['Neighborhood'],
                                   latitudes=washington_data['Latitude'],
                                   longitudes=washington_data['Longitude']
                                  )

U Street Corridor
Logan Circle
Columbia Heights


In [27]:
print(washington_venues.shape)
washington_venues.head()

(211, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,U Street Corridor,38.9171,-77.0291,Ben's Chili Bowl,38.917320,-77.028760,Hot Dog Joint
1,U Street Corridor,38.9171,-77.0291,&pizza,38.916755,-77.029245,Pizza Place
2,U Street Corridor,38.9171,-77.0291,Izakaya Seki,38.918151,-77.027777,Japanese Restaurant
3,U Street Corridor,38.9171,-77.0291,U Street Music Hall,38.917145,-77.027727,Music Venue
4,U Street Corridor,38.9171,-77.0291,Busboys and Poets,38.917909,-77.031680,American Restaurant


In [28]:
washington_venues.groupby('Neighborhood')['Venue'].count()

Neighborhood
Columbia Heights      47
Logan Circle          64
U Street Corridor    100
Name: Venue, dtype: int64

In [29]:
washington_venues.groupby(['Neighborhood','Venue Category'])['Venue'].count()

Neighborhood       Venue Category                 
Columbia Heights   American Restaurant                1
                   Art Gallery                        1
                   Asian Restaurant                   1
                   Bakery                             3
                   Bar                                3
                   Café                               1
                   Caribbean Restaurant               1
                   Coffee Shop                        1
                   College Administrative Building    1
                   Cuban Restaurant                   1
                   Discount Store                     1
                   Donut Shop                         1
                   Electronics Store                  1
                   Ethiopian Restaurant               1
                   Farmers Market                     1
                   Fast Food Restaurant               1
                   Filipino Restaurant               

In [39]:
# one hot encoding
washington_onehot = pd.get_dummies(washington_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
washington_onehot['Neighborhood'] = washington_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [washington_onehot.columns[-1]] + list(washington_onehot.columns[:-1])
washington_onehot = washington_onehot[fixed_columns]


In [31]:
washington_onehot.shape

(211, 103)

In [40]:
washington_grouped = washington_onehot.groupby('Neighborhood').mean().reset_index()

In [33]:
washington_grouped.shape

(3, 103)

In [34]:
num_top_venues = 5

for hood in washington_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = washington_grouped[washington_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Columbia Heights----
                   venue  freq
0                 Bakery  0.06
1  Vietnamese Restaurant  0.06
2                    Bar  0.06
3            Pizza Place  0.06
4            Music Venue  0.04


----Logan Circle----
                 venue  freq
0  American Restaurant  0.08
1         Cocktail Bar  0.05
2   Italian Restaurant  0.05
3          Coffee Shop  0.05
4             Wine Bar  0.03


----U Street Corridor----
                 venue  freq
0                  Bar  0.07
1  American Restaurant  0.04
2           Restaurant  0.04
3           Taco Place  0.03
4          Coffee Shop  0.03




In [35]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [36]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = washington_grouped['Neighborhood']

for ind in np.arange(washington_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(washington_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Columbia Heights,Bakery,Vietnamese Restaurant,Pizza Place,Bar,Music Venue,Coffee Shop,Caribbean Restaurant,Ethiopian Restaurant,Pet Store,Park
1,Logan Circle,American Restaurant,Italian Restaurant,Coffee Shop,Cocktail Bar,Wine Bar,New American Restaurant,Thai Restaurant,Sushi Restaurant,Ethiopian Restaurant,Gay Bar
2,U Street Corridor,Bar,Restaurant,American Restaurant,Taco Place,Mexican Restaurant,New American Restaurant,Coffee Shop,Pizza Place,Cocktail Bar,Southern / Soul Food Restaurant


In [37]:
# create map of New York using latitude and longitude values
map_washington2 = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, neighborhood in zip(washington_venues['Venue Latitude'], washington_venues['Venue Longitude'], washington_venues['Neighborhood']):
    label = '{}'.format(neighborhood)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_washington2)  
    
map_washington2